In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np
import time

In [2]:
start = time.time()
# 인기순 정렬
targetSite = 'https://series.naver.com/novel/categoryProductList.series?categoryTypeCode=all'
driver = webdriver.Chrome()
driver.get(targetSite)
wait = WebDriverWait(driver, 10)

#//*[@id="content"]/p/a[1] # 2
#//*[@id="content"]/p/a[2] # 3
#//*[@id="content"]/p/a[3] # 4
#//*[@id="content"]/p/a[4] # 5
#//*[@id="content"]/p/a[5] # 6
#//*[@id="content"]/p/a[6] # 7
#//*[@id="content"]/p/a[6] # 8
#//*[@id="content"]/p/a[6] # 9


# 페이지 수에 따라 설정
total_pages = 5000 // 25 # 400 페이지 10.000개

Titles = []  # 제목 저장할 리스트
Genre = []  # 장르 저장할 리스트
Author = []  # 작가 저장할 리스트
Start = []  # 연재 시작일
Last = []  # 마지막 연재일
Count = []  # 연재수
Good = []  # 추천수
Point = []  # 평점
Comment = []  # 댓글 수
n=0
for page in range(1, total_pages + 1):
    #print(page)
    
    # 25개의 목록을 순차적으로 클릭
    for i in range(1, 26):
        # 각 목록의 XPath
        link_xpath = f'//*[@id="content"]/div/ul/li[{i}]/div/h3/a' # 페이지별 1~25번 링크
        
        # 크롤링 작업 수행
        try:
            # 목록 클릭
            element = driver.find_element(By.XPATH, link_xpath)
            element.click()
            
            if "login" in driver.current_url or "adult" in driver.current_url: # 19금, 로그인 필요한 경우
                # 이후 전처리 할 떄 쉽게 하도록 19금 통일
                Titles.append("19금")
                Genre.append("19금")
                Author.append("19금")
                Point.append("19금")
                Start.append("19금")
                Last.append("19금")
                Count.append("19금")
                Good.append("19금")
                Comment.append("19금")
                print(f'{n+1}번, 19금입니다.')
                n += 1
                driver.back()
                continue      
            
            if "stopSale" in driver.current_url: # 오래되서 판매 중지됨
                Titles.append("판매중지")
                Genre.append("판매중지")
                Author.append("판매중지")
                Point.append("판매중지")
                Start.append("판매중지")
                Last.append("판매중지")
                Count.append("판매중지")
                Good.append("판매중지")
                Comment.append("판매중지")
                print(f'{n+1}번, 판매 중지 되었습니다.')
                n += 1
                driver.back()
                continue
            
            # 연재 시작일과 마지막 연재일 수집
            # 정렬 기준이 랜덤이라 Start, Last에 들어가는게 반대로 들어가는 경우가 있음 (최신순, 1화부터)
            a = driver.find_element(By.XPATH, '//*[@id="content"]/table/thead/tr/th[1]/div')  # 최신순
            b = driver.find_element(By.XPATH, '//*[@id="content"]/table/thead/tr/th[2]/div')  # 1화부터
            
            if a.get_attribute('class') == 'on _changeTicketSortOrder(DESC)':  # 최신순 정렬
                wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="volumeList"]/tr[1]/td[1]/em')))
                Last.append(driver.find_element(By.XPATH, '//*[@id="volumeList"]/tr[1]/td[1]/em').text)
                b.click()
                wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="volumeList"]/tr[1]/td[1]/em')))
                Start.append(driver.find_element(By.XPATH, '//*[@id="volumeList"]/tr[1]/td[1]/em').text)
                driver.back()
            
            elif b.get_attribute('class') == 'on _changeTicketSortOrder(ASC)':  # 1화부터 정렬
                wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="volumeList"]/tr[1]/td[1]/em')))
                Start.append(driver.find_element(By.XPATH, '//*[@id="volumeList"]/tr[1]/td[1]/em').text)
                a.click()
                wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="volumeList"]/tr[1]/td[1]/em')))
                Last.append(driver.find_element(By.XPATH, '//*[@id="volumeList"]/tr[1]/td[1]/em').text)
                driver.back()
            
            try:
                Titles.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/h2').text)
                Good.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div[2]/ul/li[2]/div/a/em').text)
            except:
                Titles.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/h2').text)
                Good.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[2]/ul/li[2]/div/a/em').text)
            
            try:
                Point.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div[1]/em').text)
            except:
                Point.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/em').text)
            
            Genre.append(driver.find_element(By.XPATH, '//*[@id="content"]/ul[1]/li/ul/li[2]/span/a').text)
            Author.append(driver.find_element(By.XPATH, '//*[@id="content"]/ul[1]/li/ul/li[3]/a').text)
            Count.append(driver.find_element(By.XPATH, '//*[@id="content"]/h5/strong').text)
            
            wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="cbox_module"]/div/div[1]/span')))
            Comment.append(driver.find_element(By.XPATH, '//*[@id="cbox_module"]/div/div[1]/span').text) 
            
            
            print(f'{n+1}번, page:{page}. 제목: {Titles[n]}, 장르: {Genre[n]}, 작가: {Author[n]}, 시작일: {Start[n]}, 최신화: {Last[n]}, 연재수: {Count[n]}, 추천수: {Good[n]}, 평점: {Point[n]}, 댓글: {Comment[n]}')
            n += 1
            driver.back()
            
            # 페이지 로드 대기 (필요에 따라 조정)
            time.sleep(1)
        
        except Exception as e:
            print(f"Error at page {page}, item {i}: {e}")
            break
    # for 1~25번 반복 ======================================================
    
    # 다음 페이지로 이동
    next_link_xpath = f'//*[@id="content"]/p/a[{6 if page >= 6 else page}]'
    
    try:
        next_page_element = driver.find_element(By.XPATH, next_link_xpath)
        next_page_element.click()
        
        # 페이지 로드 대기 (필요에 따라 조정)
        time.sleep(0.5)
    
    except Exception as e:
        #print(f"Error navigating to next page {page}: {e}")
        break
        
# 드라이버 종료
#driver.quit()

end = time.time()


1번, page:1. 제목: 불로불사로 산다는 것은, 장르: 판타지, 작가: 겨울에어컨, 시작일: (2023.11.29.), 최신화: (2024.06.27.), 연재수: 196, 추천수: 23, 평점: 8.8, 댓글: 1,710
2번, page:1. 제목: 천마홈즈 런던앙복 [독점], 장르: 무협, 작가: 옴니버, 시작일: (2023.03.28.), 최신화: (2024.06.27.), 연재수: 394, 추천수: 125, 평점: 8.7, 댓글: 5,977
3번, page:1. 제목: 이름대로 살련다, 장르: 현판, 작가: SSANTA, 시작일: (2023.12.06.), 최신화: (2024.06.27.), 연재수: 151, 추천수: 2, 평점: 9.3, 댓글: 40
4번, page:1. 제목: 부패의 사제, 장르: 판타지, 작가: 사다듬, 시작일: (2020.12.17.), 최신화: (2024.06.27.), 연재수: 375, 추천수: 116, 평점: 8.8, 댓글: 3,587
5번, page:1. 제목: 불로불사가 사는 법 [독점], 장르: 현판, 작가: Neuf, 시작일: (2023.01.24.), 최신화: (2024.06.27.), 연재수: 409, 추천수: 30, 평점: 9.3, 댓글: 4,169
6번, page:1. 제목: 턴제의 마법사, 장르: 판타지, 작가: 정따블, 시작일: (2024.05.17.), 최신화: (2024.06.27.), 연재수: 139, 추천수: 9, 평점: 9.6, 댓글: 1,126
7번, page:1. 제목: 파브르 in 사천당가, 장르: 무협, 작가: 에르훗, 시작일: (2024.01.09.), 최신화: (2024.06.27.), 연재수: 273, 추천수: 22, 평점: 9.6, 댓글: 5,278
8번, page:1. 제목: 칭찬스티커 받는 단종, 장르: 판타지, 작가: 문환, 시작일: (2024.02.28.), 최신화: (2024.06.27.), 연재수: 186, 추천수: 6, 평점: 9.8, 댓글: 1,731
9번, pag

66번, page:3. 제목: 파혼 후 S급 능력을 얻었다, 장르: 현판, 작가: 뭉낑깡, 시작일: (2024.05.20.), 최신화: (2024.06.27.), 연재수: 136, 추천수: 1, 평점: 5.1, 댓글: 120
67번, page:3. 제목: 담씨세가 소공자는 사파제일인 [독점], 장르: 무협, 작가: 하긴, 시작일: (2024.01.02.), 최신화: (2024.06.27.), 연재수: 189, 추천수: 17, 평점: 9.1, 댓글: 249
68번, page:3. 제목: 등선 대신 귀환했다 [독점], 장르: 현판, 작가: 담인, 시작일: (2023.11.14.), 최신화: (2024.06.27.), 연재수: 184, 추천수: 2, 평점: 6.9, 댓글: 99
69번, page:3. 제목: 메이저리그로 간다, 장르: 현판, 작가: 마인네스, 시작일: (2024.04.19.), 최신화: (2024.06.27.), 연재수: 170, 추천수: 2, 평점: 7.3, 댓글: 157
70번, page:3. 제목: 격랑을 누르는 검, 장르: 무협, 작가: 백연, 시작일: (2024.05.31.), 최신화: (2024.06.27.), 연재수: 133, 추천수: 2, 평점: 8.7, 댓글: 194
71번, page:3. 제목: 차라리 빌런으로 살겠다, 장르: 현판, 작가: 서인하, 시작일: (2024.06.20.), 최신화: (2024.06.27.), 연재수: 129, 추천수: 10, 평점: 9.7, 댓글: 539
72번, page:3. 제목: 천산방 [독점], 장르: 무협, 작가: 허담, 시작일: (2023.12.19.), 최신화: (2024.06.27.), 연재수: 214, 추천수: 9, 평점: 9.0, 댓글: 845
73번, page:3. 제목: 똥손 재벌 3세 회귀하다, 장르: 현판, 작가: Primus, 시작일: (2024.05.06.), 최신화: (2024.06.27.), 연재수: 153, 추천수: 3, 평점: 7.9, 댓글: 167
74번, page:

130번, page:6. 제목: 중간보스의 심복으로 살아남기 [독점], 장르: 판타지, 작가: 여유의빛, 시작일: (2023.10.31.), 최신화: (2024.06.27.), 연재수: 211, 추천수: 35, 평점: 9.6, 댓글: 2,290
131번, page:6. 제목: 마(魔)의 걸음은 무겁다 [독점], 장르: 무협, 작가: 검게빛남, 시작일: (2024.02.20.), 최신화: (2024.06.27.), 연재수: 197, 추천수: 4, 평점: 8.8, 댓글: 100
132번, page:6. 제목: 랭커들의 스승이 되었습니다 [독점], 장르: 현판, 작가: 정혜성, 시작일: (2024.03.12.), 최신화: (2024.06.27.), 연재수: 153, 추천수: 5, 평점: 8.9, 댓글: 228
133번, page:6. 제목: 마계에서 귀환한 망나니, 장르: 판타지, 작가: 천상아, 시작일: (2023.06.15.), 최신화: (2024.06.27.), 연재수: 385, 추천수: 2, 평점: 9.2, 댓글: 25
134번, page:6. 제목: 당신을 사랑하는 윈프린턴 [선공개], 장르: 로판, 작가: 이리내, 시작일: (2024.05.15.), 최신화: (2024.06.27.), 연재수: 111, 추천수: 4, 평점: 9.1, 댓글: 19
135번, page:6. 제목: 머릿속에 천마가 박제됨 [독점], 장르: 판타지, 작가: 찰하리, 시작일: (2023.08.22.), 최신화: (2024.06.27.), 연재수: 313, 추천수: 9, 평점: 9.0, 댓글: 1,823
136번, page:6. 제목: 창룡군림 [독점], 장르: 무협, 작가: 북미혼, 시작일: (2023.09.26.), 최신화: (2024.06.27.), 연재수: 291, 추천수: 18, 평점: 7.7, 댓글: 2,175
137번, page:6. 제목: 나 혼자 히든농장 [독점], 장르: 현판, 작가: 한얼23, 시작일: (2023.10.24.), 최신화: (2024.06.27.),

195번, page:8. 제목: 마교에서 도망친 천마, 장르: 무협, 작가: 단산자, 시작일: (2024.01.18.), 최신화: (2024.06.27.), 연재수: 255, 추천수: 1, 평점: 8.4, 댓글: 20
196번, page:8. 제목: 무림진격, 장르: 무협, 작가: 홍정표, 시작일: (2024.03.21.), 최신화: (2024.06.27.), 연재수: 210, 추천수: 좋아요, 평점: 7.6, 댓글: 1
197번, page:8. 제목: 대막리지 연개소문, 장르: 판타지, 작가: 송명관, 시작일: (2024.05.23.), 최신화: (2024.06.27.), 연재수: 165, 추천수: 2, 평점: 7.5, 댓글: 47
198번, page:8. 제목: 건달이 각성하면 벌어지는 일, 장르: 현판, 작가: 널향달, 시작일: (2024.01.25.), 최신화: (2024.06.27.), 연재수: 250, 추천수: 2, 평점: 7.3, 댓글: 13
199번, page:8. 제목: 몰락한 남작가의 창술천재 [독점], 장르: 판타지, 작가: 섬에서, 시작일: (2024.04.23.), 최신화: (2024.06.27.), 연재수: 75, 추천수: 6, 평점: 8.3, 댓글: 42
200번, page:8. 제목: 삼국지-태평비술전, 장르: 판타지, 작가: 푸달, 시작일: (2024.06.27.), 최신화: (2024.06.27.), 연재수: 204, 추천수: 좋아요, 평점: 0.0, 댓글: 0
201번, page:9. 제목: 무기 키우는 플레이어 [독점], 장르: 현판, 작가: 유다, 시작일: (2024.02.27.), 최신화: (2024.06.27.), 연재수: 151, 추천수: 좋아요, 평점: 6.9, 댓글: 96
202번, page:9. 제목: S급 짐꾼의 슬기로운 성좌 사냥 생활 [독점], 장르: 판타지, 작가: SIDCROSS, 시작일: (2024.04.02.), 최신화: (2024.06.27.), 연재수: 106, 추천수: 8, 평점: 9.0, 댓글

269번, page:11. 제목: 괴물 대공가에 납치된 것 같습니다 [독점], 장르: 로판, 작가: 칸나꽃, 시작일: (2024.03.08.), 최신화: (2024.06.27.), 연재수: 126, 추천수: 2, 평점: 8.9, 댓글: 152
270번, page:11. 제목: 보험왕이 장보고를 업어 키움 [독점], 장르: 판타지, 작가: 두루미야, 시작일: (2024.04.09.), 최신화: (2024.06.27.), 연재수: 123, 추천수: 20, 평점: 9.3, 댓글: 64
271번, page:11. 제목: 천마를 베고 회귀 탈출 [독점], 장르: 무협, 작가: 천영, 시작일: (2024.05.07.), 최신화: (2024.06.27.), 연재수: 61, 추천수: 1, 평점: 8.1, 댓글: 28
272번, page:11. 제목: 꿀벌을 주웠더니 각성했다 [무료연재], 장르: 현판, 작가: 신광호, 시작일: (2024.05.21.), 최신화: (2024.06.27.), 연재수: 47, 추천수: 4, 평점: 9.2, 댓글: 359
273번, page:11. 제목: 빼앗긴 마법명가의 환생 가주 [무료연재], 장르: 판타지, 작가: 피키콜드, 시작일: (2024.06.11.), 최신화: (2024.06.27.), 연재수: 26, 추천수: 14, 평점: 9.0, 댓글: 64
274번, page:11. 제목: 암흑가에서 눈을 뜬 소드마스터 [무료연재], 장르: 판타지, 작가: 동경, 시작일: (2024.06.25.), 최신화: (2024.06.27.), 연재수: 12, 추천수: 9, 평점: 8.9, 댓글: 18
275번, page:11. 제목: 무림공적으로 살아남기 [독점], 장르: 무협, 작가: 기신, 시작일: (2023.02.07.), 최신화: (2024.06.27.), 연재수: 458, 추천수: 4, 평점: 9.1, 댓글: 259
276번, page:12. 제목: 천마는 평범하게 살 수 없다 [독점], 장르: 판타지, 작가: 산천, 시작일: (2020.09.

...

4536번, page:182. 제목: 나 혼자 게이트 속 세상을 알고 있다., 장르: 판타지, 작가: 은퇴생활, 시작일: (2024.04.09.), 최신화: (2024.04.09.), 연재수: 130, 추천수: 좋아요, 평점: 8.1, 댓글: 40
4537번, page:182. 제목: 절대자의 세계 [독점], 장르: 현판, 작가: 까마귀씨, 시작일: (2023.06.06.), 최신화: (2024.04.09.), 연재수: 270, 추천수: 3, 평점: 8.7, 댓글: 258
4538번, page:182. 제목: 이번 생에는 물리력 만렙 변호사로 살겠습니다, 장르: 현판, 작가: 백은빛별, 시작일: (2023.11.23.), 최신화: (2024.04.09.), 연재수: 202, 추천수: 2, 평점: 9.5, 댓글: 64
4539번, page:182. 제목: 전직 절대자는 아카데미 펫 관리자 [독점], 장르: 현판, 작가: 말랑부들, 시작일: (2023.06.06.), 최신화: (2024.04.09.), 연재수: 238, 추천수: 17, 평점: 9.5, 댓글: 947
4540번, page:182. 제목: 일식이가 간다 [단행본], 장르: 현판, 작가: 이후, 시작일: (2013.12.05.), 최신화: (2024.04.09.), 연재수: 69, 추천수: 71, 평점: 5.9, 댓글: 141
4541번, page:182. 제목: 나 혼자만 SSS급 약팔이 헌터, 장르: 판타지, 작가: 상연, 시작일: (2023.12.01.), 최신화: (2024.04.09.), 연재수: 203, 추천수: 6, 평점: 9.3, 댓글: 840
4542번, 19금입니다.
4543번, page:182. 제목: 마왕은 슈퍼스타, 장르: 현판, 작가: 수라백, 시작일: (2024.03.14.), 최신화: (2024.04.09.), 연재수: 150, 추천수: 2, 평점: 8.4, 댓글: 55
4544번, page:182. 제목: 남편을 잘못 키워 버렸다, 장르: 로판, 작가: 다라빛, 시작일: 

4644번, 19금입니다.
4645번, 19금입니다.
4646번, 19금입니다.
4647번, 19금입니다.
4648번, 19금입니다.
4649번, 19금입니다.
4650번, 19금입니다.
4651번, 19금입니다.
4652번, 19금입니다.
4653번, page:187. 제목: 세상에 나쁜 초월자는 없다, 장르: 로판, 작가: 윤달(YounDal), 시작일: (2024.04.06.), 최신화: (2024.04.06.), 연재수: 146, 추천수: 좋아요, 평점: 9.4, 댓글: 27
4654번, page:187. 제목: 천재 배우, 이혼 후 다시 태어나다 [단행본], 장르: 현판, 작가: 마파두부, 시작일: (2024.04.06.), 최신화: (2024.04.06.), 연재수: 5, 추천수: 좋아요, 평점: 0.0, 댓글: 0
4655번, page:187. 제목: 악당들의 최애는 나, 장르: 로판, 작가: 윤온, 시작일: (2024.04.06.), 최신화: (2024.04.06.), 연재수: 166, 추천수: 좋아요, 평점: 9.5, 댓글: 22
4656번, 19금입니다.
4657번, 19금입니다.
4658번, 19금입니다.
4659번, 19금입니다.
4660번, page:187. 제목: 악녀님 빵 구워주세용, 장르: 로판, 작가: 연백림, 시작일: (2024.04.06.), 최신화: (2024.04.06.), 연재수: 137, 추천수: 좋아요, 평점: 9.9, 댓글: 17
4661번, page:187. 제목: 입조심하세요, 성녀님!, 장르: 로판, 작가: 하이마이디어, 시작일: (2024.04.06.), 최신화: (2024.04.06.), 연재수: 173, 추천수: 좋아요, 평점: 7.9, 댓글: 2
4662번, page:187. 제목: 폭군에게 처형당하는 황후가 되었습니다, 장르: 로판, 작가: 수비, 시작일: (2024.04.06.), 최신화: (2024.04.06.), 연재수: 152, 추천수: 1, 평점: 9.1, 댓글: 24
4663번, page:1

4744번, page:190. 제목: 조선의 혁명을 위하여, 장르: 판타지, 작가: 매검향, 시작일: (2024.04.04.), 최신화: (2024.04.04.), 연재수: 103, 추천수: 좋아요, 평점: 6.5, 댓글: 11
4745번, page:190. 제목: 파멸 속에서 나를 구원하소서 [단행본], 장르: 로판, 작가: 하이픈, 시작일: (2024.04.04.), 최신화: (2024.04.04.), 연재수: 5, 추천수: 좋아요, 평점: 0.0, 댓글: 0
4746번, page:190. 제목: 제 이능은 특별합니다, 장르: 판타지, 작가: 하마루, 시작일: (2023.05.15.), 최신화: (2024.04.04.), 연재수: 317, 추천수: 2, 평점: 8.9, 댓글: 4
4747번, page:190. 제목: 내 마력이 안 줄어듦 [단행본], 장르: 현판, 작가: 긍정론적, 시작일: (2024.04.04.), 최신화: (2024.04.04.), 연재수: 8, 추천수: 1, 평점: 10.0, 댓글: 5
4748번, page:190. 제목: 전향했더니 천재 투수가 되었다 [단행본], 장르: 현판, 작가: 작은우산, 시작일: (2024.04.04.), 최신화: (2024.04.04.), 연재수: 5, 추천수: 좋아요, 평점: 0.0, 댓글: 0
4749번, page:190. 제목: 혈음, 장르: 로맨스, 작가: 오기, 시작일: (2024.03.06.), 최신화: (2024.04.04.), 연재수: 132, 추천수: 3, 평점: 10.0, 댓글: 0
4750번, page:190. 제목: 늑대 저택의 사랑받는 새아가, 장르: 로판, 작가: 김노랑, 시작일: (2024.04.04.), 최신화: (2024.04.04.), 연재수: 187, 추천수: 1, 평점: 9.5, 댓글: 29
4751번, page:191. 제목: 황제와 공작 부인 [단행본], 장르: 로판, 작가: 이라라, 시작일: (2024.04.04.), 최신화: (2024.04.04.), 연재수: 2,

4841번, page:194. 제목: 너 따위 재벌[단행본], 장르: 로맨스, 작가: 효체, 시작일: (2024.04.03.), 최신화: (2024.04.03.), 연재수: 3, 추천수: 2, 평점: 10.0, 댓글: 0
4842번, 19금입니다.
4843번, page:194. 제목: 절대자는 신혼이다 [독점], 장르: 판타지, 작가: 현풍, 시작일: (2023.09.05.), 최신화: (2024.04.02.), 연재수: 201, 추천수: 3, 평점: 7.2, 댓글: 247
4844번, page:194. 제목: 귀신 덕에 골프 치는 회사원, 장르: 현판, 작가: 예린채린, 시작일: (2024.03.14.), 최신화: (2024.04.02.), 연재수: 140, 추천수: 1, 평점: 8.8, 댓글: 127
4845번, page:194. 제목: 우주 전쟁의 검성이 되었다 [단행본], 장르: 판타지, 작가: 이나르, 시작일: (2024.04.02.), 최신화: (2024.04.02.), 연재수: 8, 추천수: 좋아요, 평점: 10.0, 댓글: 1
4846번, page:194. 제목: 일제를 쏴라! [단행본], 장르: 판타지, 작가: 언행일치, 시작일: (2024.04.02.), 최신화: (2024.04.02.), 연재수: 13, 추천수: 좋아요, 평점: 10.0, 댓글: 1
4847번, page:194. 제목: 1905 회귀재벌, 천조국 정벌기 [단행본], 장르: 판타지, 작가: 이령, 시작일: (2024.04.02.), 최신화: (2024.04.02.), 연재수: 16, 추천수: 좋아요, 평점: 0.0, 댓글: 0
4848번, page:194. 제목: 완벽한 결혼을 거절하는 법 [단행본], 장르: 로판, 작가: 포르토피노, 시작일: (2024.04.02.), 최신화: (2024.04.02.), 연재수: 5, 추천수: 좋아요, 평점: 9.7, 댓글: 0
4849번, page:194. 제목: 지금 이 순간 [외전포함] [합본], 장르: 로맨스, 작가: 이채영, 시작일

4936번, page:198. 제목: 아내 약탈 [선공개], 장르: 로맨스, 작가: 이헤이, 시작일: (2024.03.22.), 최신화: (2024.04.01.), 연재수: 75, 추천수: 4, 평점: 9.3, 댓글: 652
4937번, page:198. 제목: 불건전한 오해, 장르: 로맨스, 작가: 오사라, 시작일: (2024.04.01.), 최신화: (2024.04.01.), 연재수: 75, 추천수: 좋아요, 평점: 9.7, 댓글: 22
4938번, page:198. 제목: 다정한 개새끼를 버리겠습니다, 장르: 로판, 작가: 댕휘, 시작일: (2024.04.01.), 최신화: (2024.04.01.), 연재수: 178, 추천수: 좋아요, 평점: 0.0, 댓글: 0
4939번, page:198. 제목: 이번엔 진짜 남주인가요 [선공개], 장르: 로판, 작가: 은후C, 시작일: (2024.02.23.), 최신화: (2024.04.01.), 연재수: 105, 추천수: 9, 평점: 9.5, 댓글: 127
4940번, 19금입니다.
4941번, page:198. 제목: 보통의 연애 [외전증보판], 장르: 로맨스, 작가: 사란, 시작일: (2024.04.01.), 최신화: (2024.04.01.), 연재수: 55, 추천수: 좋아요, 평점: 10.0, 댓글: 0
4942번, page:198. 제목: 일원분천 [단행본], 장르: 무협, 작가: 형상준, 시작일: (2024.04.01.), 최신화: (2024.04.01.), 연재수: 8, 추천수: 좋아요, 평점: 0.0, 댓글: 0
4943번, page:198. 제목: 악역 영애지만, 정원사로 전직하려 합니다 [외전 선공개], 장르: 로판, 작가: 박둥찌, 시작일: (2024.04.01.), 최신화: (2024.04.01.), 연재수: 179, 추천수: 1, 평점: 10.0, 댓글: 0
4944번, page:198. 제목: 링크 [단행본], 장르: 현판, 작가: 박재학, 시작일: (2024.04.01.), 최신화: (202

In [3]:
df = pd.DataFrame({"제목": Titles, "장르": Genre, "작가": Author, "연재 시작일": Start, "마지막 연재일": Last, "회수": Count, "댓글수": Comment, "추천수": Good, "평점": Point})
df.index = df.index + 1

In [4]:
df.to_csv('./data/NSeries_최신순_5,000.txt')

In [5]:
print(end-start) # 20973.018662452698 대략 6시간 정도 걸림

20973.018662452698
